In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

dataset = pd.read_csv("../dataset/diabetes.csv")

X = dataset.iloc[:, :-1]
y = dataset.iloc[:, -1]

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
n_splits = 5
indices = np.arange(len(X))
np.random.shuffle(indices)

fold_indices = np.array_split(indices, n_splits)


for i in range(n_splits):
    test_indices = fold_indices[i]
    train_indices = np.concatenate(fold_indices[:i] + fold_indices[i+1:])

    X_train, X_test = X.iloc[train_indices], X.iloc[test_indices]
    y_train, y_test = y.iloc[train_indices], y.iloc[test_indices]

X_train = (X_train - X_train.mean()) / X_train.std()
X_test = (X_test - X_test.mean()) / X_test.std()

def knn_predict(X_train, y_train, x_test, k=5):
    distances = np.sqrt(np.sum((X_train - x_test) ** 2, axis = 1))
    indices = np.argsort(distances)[:k]
    neighbors = y_train.iloc[indices]
    return np.bincount(neighbors).argmax()

y_pred = [knn_predict(X_train, y_train, x_test) for _, x_test in X_test.iterrows()]

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(accuracy * 100))

Accuracy: 69.93%


In [2]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
tp = cm[0, 0]
fp = cm[0, 1]
fn = cm[1, 0]
tn = cm[1, 1]
accuracy=(tp+tn)/(tp+fp+fn+tn)
print(accuracy)

0.6993464052287581
